<img src="https://raw.githubusercontent.com/imohealth/solution-engineering/refs/heads/updates/RWE-Cohort-Identification/PythonNotebooks/PatientData-To-OMOP-And-Cohort-Identification/static/imo_health.png?token=GHSAT0AAAAAADSTDGZJSSJF42RDYZPNU2YY2LYZN7Q" alt="IMO Health Logo" width="300"/>

---

# RWE Patient Data Extraction from Clinical Notes

This notebook is the **first step** in the RWE Cohort Identification pipeline. It processes unstructured clinical notes through IMO's NLP API to extract structured patient data.

## Workflow Overview

This notebook performs NLP-based entity extraction from IHM (Intelligent Health Management) clinical notes:

1. **CSV Data Loading**: Load unstructured clinical notes from CSV files in `upload_ihm` folder
2. **API Authentication**: Authenticate with IMO's API services using Auth0 credentials
3. **Input Validation**: Verify CSV structure and required fields (hospitalGuid, encounterGuid, patientGuid, notesDEID)
4. **NLP Entity Extraction**: Process notes through IMO's Clinical Comprehensive NLP pipeline to extract:
   - **Problems/Diagnoses**: Medical conditions with ICD10CM codes
   - **Medications**: Drug exposures with RXNORM codes
   - **Lab Tests**: Measurements with LOINC codes
   - **Procedures**: Clinical procedures with CPT codes
5. **Structured Output**: Generate `patient_output.xlsx` with extracted entities and standard medical codes

## Output

The extracted patient data (`patient_output.xlsx`) serves as input for the next step in the pipeline:
- **Next Step**: OMOP conversion using `Patient_Data_to_OMOP_Converter.ipynb`
- **Final Step**: RWE gneration by matching with eligibility criteria defined in IMO Precision Set Editor

## Prerequisites

- IMO API credentials configured in `config.json`
- CSV files with clinical notes in `upload_ihm/` directory
- Required Python packages: `pandas`, `openpyxl`, `xlsxwriter`, `requests`

---

## Step 1: Environment Setup and Configuration


### Verify Required Packages

This cell verifies that all necessary Python packages are installed:

- **`pandas`**: Data manipulation and analysis library
- **`openpyxl`**: Reading/writing Excel files
- **`xlsxwriter`**: Creating formatted Excel files
- **`requests`**: HTTP library for API calls to IMO services

> **Note**: These packages should already be installed in your virtual environment.

In [ ]:
import pandas as pd
import requests
import json
import os

# Install required packages (uncomment if running in a new environment)
# Install required packages (uncomment if running in a new environment)
%pip install boto3 requests  pandas  
 

---

## Step 2: Load API Configuration

### Load IMO API Credentials

This cell loads the API credentials from `config.json` and obtains an authentication token:

- **Auth0 Authentication**: Authenticates with IMO's Auth0 service
- **Bearer Token**: Retrieves access token for API calls
- **Token Storage**: Stores token globally for use in subsequent steps

The configuration file should contain:
- `client_id`: Your IMO API client ID
- `client_secret`: Your IMO API client secret
- `audience`: API audience identifier
- `domain`: Auth0 domain

This token will be used for both NLP entity extraction and FHIR valueset searches.

In [ ]:
def load_config():
    """Load configuration from config.json file"""
    # When running from this notebook (in using-OMOP), config.json is one folder up
    if '__file__' in globals():
        config_path = os.path.join(os.path.dirname(__file__), 'config.json')
    else:
        config_path = os.path.join('..', 'config.json')
    
    with open(config_path, 'r') as f:
        return json.load(f)

def get_auth0_token(config):
    """Get access token from Auth0"""
    auth0_config = config['auth0']
    
    payload = {
        'client_id': auth0_config['client_id'],
        'client_secret': auth0_config['client_secret'],
        'audience': auth0_config['audience'],
        'grant_type': 'client_credentials'
    }
    
    headers = {'content-type': 'application/json'}
    token_url = f"https://{auth0_config['domain']}/oauth/token"
    
    response = requests.post(token_url, json=payload, headers=headers)
    
    if response.status_code == 200:
        token_data = response.json()
        return token_data['access_token']
    else:
        raise Exception(f"Failed to get token: {response.status_code} - {response.text}")

# Load configuration and get token
config = load_config()
Token = get_auth0_token(config)

print("✅ Successfully authenticated with IMO API")
print(f"   Token obtained: {Token[:20]}...")

---

## Step 3: Verify Input Data

### Check CSV Files in upload_ihm Folder

This cell verifies that the required CSV files are present in the `upload_ihm` directory:

1. **Directory Check**: Ensures `upload_ihm` folder exists
2. **File Discovery**: Lists all CSV files in the directory
3. **Data Preview**: Shows sample records from the CSV file
4. **Column Validation**: Verifies required columns (hospitalGuid, encounterGuid, patientGuid, notesDEID)

**Expected Input**: CSV file with unstructured clinical notes containing:
- `hospitalGuid`: Unique hospital identifier
- `encounterGuid`: Unique encounter identifier  
- `patientGuid`: Unique patient identifier
- `notesDEID`: De-identified clinical notes text

> **Important**: Ensure your CSV files are placed in the `upload_ihm/` directory before running this cell.

In [ ]:
# Verify CSV files in upload_ihm folder
upload_folder = '../upload_notes'

if not os.path.exists(upload_folder):
    print(f"❌ Error: '{upload_folder}' directory not found!")
    print(f"   Please create the directory and place your CSV files there.")
else:
    csv_files = [f for f in os.listdir(upload_folder) if f.endswith('.csv')]
    
    if not csv_files:
        print(f"❌ No CSV files found in '{upload_folder}' directory")
    else:
        print(f"✅ Found {len(csv_files)} CSV file(s) in '{upload_folder}':")
        for csv_file in csv_files:
            file_path = os.path.join(upload_folder, csv_file)
            print(f"\n📁 File: {csv_file}")
            
            # Load and preview the CSV
            try:
                df = pd.read_csv(file_path)
                print(f"   Records: {len(df)}")
                print(f"   Columns: {list(df.columns)}")
                
                # Check for required columns
                required_cols = ['hospitalGuid', 'encounterGuid', 'patientGuid', 'notesDEID']
                missing_cols = [col for col in required_cols if col not in df.columns]
                
                if missing_cols:
                    print(f"   ⚠️ Warning: Missing columns: {missing_cols}")
                else:
                    print(f"   ✅ All required columns present")
                    
                # Show sample data
                print(f"\n   Sample records:")
                print(df.head(3).to_string(index=False, max_colwidth=50))
                
            except Exception as e:
                print(f"   ❌ Error reading file: {e}")

---

## Step 4: NLP Entity Extraction - Core Processing

### Extract Structured Patient Data from Unstructured Clinical Notes

This is the **core processing step** that transforms unstructured IHM clinical notes into structured patient data using IMO's NLP API.

#### Processing Workflow:

1. **Input Validation**: Reads and validates CSV file from `upload_ihm` folder
2. **Note-by-Note Processing**: Iterates through each patient note (notesDEID column)
3. **NLP API Calls**: Sends clinical text to IMO's Clinical Comprehensive pipeline
4. **Entity Recognition**: Uses advanced NLP to identify:
   - Medical problems and diagnoses
   - Medications and drug exposures
   - Laboratory tests and results
   - Clinical procedures
5. **Code Mapping**: Automatically maps extracted entities to standard medical vocabularies:
   - **ICD10CM**: International Classification of Diseases codes for conditions
   - **RXNORM**: Standardized medication codes
   - **LOINC**: Logical Observation Identifiers for lab tests
   - **CPT**: Current Procedural Terminology codes
6. **GUID Preservation**: Maintains linkage via hospitalGuid, encounterGuid, and patientGuid
7. **Structured Output**: Creates `patient_output.xlsx` in the `Output` folder

#### Key Features:

- **Rate Limiting**: Built-in 0.5s delay between API calls to prevent throttling
- **Error Handling**: Automatic retry logic for rate limit errors (429 responses)
- **Progress Tracking**: Real-time progress updates every 10 records
- **Data Quality**: Preserves original GUIDs for downstream OMOP conversion

#### Output File Structure (`patient_output.xlsx`):
- **Column A-C**: hospitalGuid, encounterGuid, patientGuid (preserved from input)
- **Subsequent Columns**: Extracted medical entities with:
  - Entity text as it appears in the note
  - Standard medical codes (ICD10CM, RXNORM, LOINC, CPT)
  - Entity descriptions and clinical context
  - Semantic tags (problem, medication, lab, procedure)

> **Important**: This extracted patient data file is required for the next step (OMOP conversion). The `Patient_Data_to_OMOP_Converter.ipynb` notebook will read this file and transform it into OMOP CDM v6.0 format.

> **Note**: Processing time depends on the number of records. With 0.5s delay per record, expect approximately 25-30 seconds for 50 records.


In [ ]:
# Run NLP Entity Extraction Process
import time
import sys
import os

# Ensure we can import shared.py from the parent folder
try:
    from shared import create_workbook
except ModuleNotFoundError:
    sys.path.insert(0, os.path.abspath('..'))
    from shared import create_workbook

print("🚀 Starting NLP Entity Extraction Process...")
print("=" * 60)

# Configuration
csv_file_path = '../upload_ihm/unstructured_IHM.csv'
output_workbook_name = 'patient_output.xlsx'  # create_workbook adds Output/ prefix
url = 'https://api.imohealth.com/entityextraction/pipelines/imo-clinical-comprehensive?version=3.0'
domain_filter = ["problem", "procedure", "medication", "lab"]
score_threshold = 0.0
request_delay = 0.5  # seconds between API calls to avoid rate limiting

def invoke_api(file_contents, auth_token):
    """Call IMO NLP API for entity extraction"""
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {auth_token}"
    }
    request_body = {
        "text": file_contents,
        "preferences": {
            "thresholds": {"global": score_threshold},
            "domain_filter": domain_filter
        },
    }
    response = requests.post(url, data=json.dumps(request_body), headers=headers)
    response.raise_for_status()
    return response

try:
    start_time = time.time()
    
    # Check if CSV file exists
    if not os.path.exists(csv_file_path):
        print(f"❌ Error: CSV file not found at {csv_file_path}")
        print(f"   Please ensure the file exists in the upload_ihm folder")
    else:
        # Read CSV file
        print(f"📁 Reading CSV file: {csv_file_path}")
        df = pd.read_csv(csv_file_path)
        print(f"   Total records to process: {len(df)}")
        
        # Verify required columns
        required_cols = ['hospitalGuid', 'encounterGuid', 'patientGuid', 'notesDEID']
        missing_cols = [col for col in required_cols if col not in df.columns]
        
        if missing_cols:
            print(f"❌ Error: Missing required columns: {missing_cols}")
        else:
            # Process each note through the API
            api_responses = {}  # Dictionary format expected by create_workbook
            
            print(f"\n🔄 Processing notes through IMO NLP API...")
            print(f"   (Adding {request_delay}s delay between requests to avoid rate limiting)")
            
            for idx, row in df.iterrows():
                note_text = row['notesDEID']
                hospital_guid = row['hospitalGuid']
                encounter_guid = row['encounterGuid']
                patient_guid = row['patientGuid']
                
                # Create unique identifier for this record
                record_id = f"record_{idx+1}"
                
                try:
                    # Call API
                    api_start = time.time()
                    response = invoke_api(note_text, Token)
                    api_elapsed = time.time() - api_start
                    
                    response_data = response.json()
                    
                    # Store response in format expected by create_workbook
                    api_responses[record_id] = {
                        'payload': response_data,
                        'hospitalGuid': hospital_guid,
                        'encounterGuid': encounter_guid,
                        'patientGuid': patient_guid,
                        'run_time': f"{api_elapsed:.2f}s"
                    }
                    
                    if (idx + 1) % 10 == 0:
                        print(f"   Processed {idx + 1}/{len(df)} records...")
                    
                    # Add delay to avoid rate limiting
                    time.sleep(request_delay)
                        
                except Exception as e:
                    error_msg = str(e)
                    if "429" in error_msg:
                        print(f"   ⚠️ Rate limit reached at record {idx + 1}. Waiting 2 seconds...")
                        time.sleep(2)
                        # Retry once
                        try:
                            response = invoke_api(note_text, Token)
                            response_data = response.json()
                            api_responses[record_id] = {
                                'payload': response_data,
                                'hospitalGuid': hospital_guid,
                                'encounterGuid': encounter_guid,
                                'patientGuid': patient_guid,
                                'run_time': 'retry'
                            }
                            print(f"   ✅ Successfully retried record {idx + 1}")
                        except Exception as retry_error:
                            print(f"   ❌ Retry failed for record {idx + 1}: {str(retry_error)}")
                    else:
                        print(f"   ⚠️ Error processing record {idx + 1}: {error_msg}")
                    continue
            
            # Create Excel workbook with results
            print(f"\n💾 Creating output workbook...")
            create_workbook(output_workbook_name, api_responses)
            
            elapsed_time = time.time() - start_time
            
            print(f"\n✅ NLP processing completed successfully!")
            print(f"   Processed records: {len(api_responses)}/{len(df)}")
            print(f"   Elapsed time: {elapsed_time:.1f} seconds")
            print(f"   Output file: Output/{output_workbook_name}")
            
            # Verify and preview output
            output_path = f'Output/{output_workbook_name}'
            if os.path.exists(output_path):
                df_output = pd.read_excel(output_path, sheet_name='Results')
                print(f"\n📊 Output Summary:")
                print(f"   Total records: {len(df_output)}")
                print(f"   Columns: {len(df_output.columns)}")
                
                if 'Semantic Tag' in df_output.columns:
                    print(f"\n   Extracted entities by type:")
                    for tag, count in df_output['Semantic Tag'].value_counts().items():
                        print(f"     {tag}: {count}")
            else:
                print(f"\n⚠️ Warning: Output file not created at {output_path}")
                
except Exception as e:
    print(f"\n❌ Error during NLP processing: {e}")
    import traceback
    traceback.print_exc()

---

## ✅ NLP Extraction Complete - Next Steps

### Successfully Extracted Patient Data

Congratulations! You have successfully extracted structured patient data from unstructured clinical notes using IMO's NLP API.

#### What You've Accomplished:

- ✅ Processed IHM clinical notes through IMO's Clinical Comprehensive NLP pipeline
- ✅ Extracted medical entities (problems, medications, labs, procedures)
- ✅ Mapped entities to standard medical vocabularies (ICD10CM, RXNORM, LOINC, CPT)
- ✅ Generated structured patient data in `Output/patient_output.xlsx`
- ✅ Preserved patient/encounter/hospital GUIDs for data linkage

#### Output File Location:
📁 **`Output/patient_output.xlsx`**

This file contains all extracted medical entities with standard codes and is ready for OMOP conversion.

---

### Next Steps in the RWE Pipeline:

#### Step 2: OMOP Conversion
**Notebook**: `Patient_Data_to_OMOP_Converter.ipynb`

Transform the extracted patient data into OMOP Common Data Model (CDM) v6.0 format:
- Convert to standardized OMOP clinical tables (PERSON, VISIT_OCCURRENCE, CONDITION_OCCURRENCE, etc.)
- Map medical codes to OMOP standard concepts
- Apply data obfuscation for privacy protection
- Generate CSV files ready for clinical data warehouses

**Input**: `Output/patient_output.xlsx` (generated by this notebook)  
**Output**: `Output/OMOP_CSV/*.csv` (OMOP CDM tables)

#### Step 3: Cohort Identification 
- Generate cohort matching reports

**Input**: `Output/OMOP_CSV/*.csv` (from Step 2)  
**Output**: `Output/cohort_matching_results.csv`

---

### File Summary:

| File | Purpose | Next Notebook |
|------|---------|---------------|
| `patient_output.xlsx` | Structured patient data with medical codes | `Patient_Data_to_OMOP_Converter.ipynb` |
| Original CSV | Source clinical notes | *(Processing complete)* |

> **Tip**: Keep the `Output/patient_output.xlsx` file - it's required for OMOP conversion in the next step!
